# Output analysis of inventory simulation model


## Table of contents

1. [Using the simulation model](#usingmodel)
2. [Output analysis of a terminating simulation](#terminatingsim)  
    2.1 [Independence of replications](#indep)  
3. [Minimizing inventory costs](#optim)  

 ## 1. Using the simulation model
 <a id="usingmodel"></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import model

To run a single iteration of the inventory simulation model, use the run function of the model.py file. User will need to enter:

- a simulation length (expressed in days) after which the simulation stops
- a reorder point specifying the inevntory level (in units) that triggers an order to replenish the stocks 
- an order size specifying the number of units to order

The function will return a dictionary containing user input parameters (reorder point, order size) and the model output (average monthly total inventory cost, ordering cost, shortage cost and holding cost).

Let's run a few iteration and analyze output:

In [13]:
# perform a single run of the inventory simulation model
model.run(length = 120., reorder_point = 20, order_size = 20)

{'reorder_point': 20,
 'order_size': 20,
 'total_cost': 125.2,
 'ordering_cost': 95.0,
 'holding_cost': 9.8,
 'shortage_cost': 20.4}

In [12]:
# run alternative configuration of the model with increased order size
model.run(length = 120., reorder_point = 20, order_size = 80)

{'reorder_point': 20,
 'order_size': 80,
 'total_cost': 126.9,
 'ordering_cost': 82.5,
 'holding_cost': 34.5,
 'shortage_cost': 9.9}

In these two iterations, we maintained the same reorder point and increased the order size from 20 to 80 units. 

This resulted in lower shortage cost (since bigger orders are placed) and lower ordering costs (orders are not placed as frequently). Holding costs predictably increased because larger inventory levels were maintained.

In terms of total costs, the second configuration isn't as good as the first one.

In [15]:
# run alternative configuration with smaller reorder point
model.run(length = 120., reorder_point = 10, order_size = 20)

{'reorder_point': 10,
 'order_size': 20,
 'total_cost': 146.7,
 'ordering_cost': 98.7,
 'holding_cost': 3.9,
 'shortage_cost': 44.2}

In this configuration, the reorder point was decrease. This reduced the average holding costs (because a smaller inventory is being maintained) at the expense of a large increase in shortage cost.

This configuration is so far the worse of the three in terms of total costs.

## 2. Output analysis of a terminating simulation

 <a id="terminatingsim"></a>
 
In this section, we'll simulate an inventory system with a 30 units reorder point and an order size of 60 units. We will estimate the avrerage monthly cost of this inventory policy after 120 months.

Since there is an event ending the simulation (after 120 months), the simulation will be analyzed as a terminating model (as opposed to analyzing steady state parameters for example).

In [2]:
# define simulation parameters
length = 120
reorder_point = 30
order_size = 60

# run independent replications
np.random.seed(1066)
first_run = model.run(length, reorder_point, order_size)
second_run = model.run(length, reorder_point, order_size)

# print results
print('First replication total costs:', first_run['total_cost'])
print('Second replication total costs:', second_run['total_cost'])

First replication total costs: 126.0
Second replication total costs: 127.2


The two independent replications of the model resulted in different total cost output, because of the randomness included in the model. 
This means that a single run will not provide a definitive measure of total inventory cost. Instead, we can run several independent replications of the simulation to build a confidence interval for total inventory costs.

###      2.1 Building a confidence interval for total inventory cost

In [3]:
np.random.seed(1066)
length = 120.
num_replications = 15

df = model.run_experiments(length, [reorder_point], [order_size], num_replications)
df

,reorder_point,order_size,total_cost,ordering_cost,holding_cost,shortage_cost
0,30,60,126.0,87.4,35.6,2.9
1,30,60,127.2,86.5,36.7,4.0
2,30,60,123.0,84.2,35.5,3.2
3,30,60,126.6,87.3,35.1,4.2
4,30,60,125.2,85.7,36.4,3.1
5,30,60,125.5,85.6,35.3,4.5
6,30,60,126.2,87.3,36.0,2.9
7,30,60,124.7,87.0,35.6,2.1
8,30,60,127.0,87.2,35.5,4.3
9,30,60,125.9,86.6,35.4,3.9


In [4]:
from scipy import stats
 
# compute sample mean and standard deviation
mean = df['total_cost'].mean()
std = df['total_cost'].std()

# compute t-statistics for a 90% confidence interval
alpha = 1-.9
tstat = stats.t.ppf(1-alpha/2, num_replications - 1)

# compute confidence interval
error_margin = tstat * std / np.sqrt(num_replications)
lbound = mean - error_margin
ubound = mean + error_margin

print("90 percent confidence interval for monthly inventory cost: [%.1f, %.1f]" % (lbound, ubound))

90 percent confidence interval for monthly inventory cost: [124.1, 126.0]


## 3. Minimizing inventory costs

<a id="optim"></a>